In [6]:
from keras.layers import Input,Dense,Reshape,Flatten,BatchNormalization,LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
import os 
import tensorflow as tf

In [7]:
# Defining image dimensions

imgRows = 28
imgCols = 28
channels = 1
imgShape = (imgRows,imgCols,channels)

In [8]:
def generator():

    """
        Creates a generator model for a Generative Adversaial Network (GAN).

        The generator takes a noise vector as input and transforms it into realistic looking image through a series of fully connected layers,Leaky Relu 
        activation , batch normalizations and reshaping. The final output is scaled tot the range [-1,1] using the 'tanh' activation function,
        making it suitable for image generation tasks.
    
        return:
            keras.Model : A keras model that maps noise vectors to generated images.
    
    """

    #Define the shape of the noise vector ; this will serve as the input to the generastor 
    # typically used in GANS, the noise vector allows the model to generate diverse outputs
    noise_shape = (100,)

    # Initilize a sequential model , which is a linear stack of layers
    model = Sequential()

    #Add a dense layer with 256 neurons, this layer acts as the first fully connected layer, transforming the 
    #input noise vector into a higher dimensional feature space.
    #the input_shape defines the expected input noise shape dimensions 
    model.add(Dense(256,input_shape = noise_shape))
    
    # Add a LekyRelu activation function with a small negative slope defined by 'alpha'
    # LeakyRelu is a variant of the standard Relu (Rectified linear unit) actibvtion function
    # While standard RELU sets the value to 0 for all the negative inputs , this allows a small, non zero gradient for negative inputs
    # This helps mitigate the "dying RELU" problem, where neurons become inactive and stop learning due to 0 gradient
    # the alpha parameter defines the slope of the activatioin function for negative inputs. A smaller alpha means means less contibution from negative values 
    # large alpha means it allows more contribution from neagtive values.
    # this activation helps prevent the 'dying relu' problem by allowing small gradient for negative inputs
    model.add(LeakyReLU(alpha = 0.2))

    # Add batch normalization layer to stablize and accelarate training by normalizing the activations of the previous layer.
    # The momentum parameter controls how much of the past running statics to use.
    # This layer also prevents internal covariate shift and improves the models generalization ability
    model.add(BatchNormalization(momentum = 0.8))

    # Add a Dense layer with 512 neurons to further expand the feature space.
    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))

    # Add another Dense layer with 1024 neurons for further expansion.
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))

    # Add the output Dense Layer to produce the final generated image.
    # np.prod(image shape) caluclates the total number of pixels (flattned shape) of the output image.
    # the activation function 'tanh' ensures that the output values are scaled bwteen -1 and 1 which is common for image generation tasks 
    model.add(Dense(np.prod(imgShape),activation = 'tanh'))

    # Reshape the output to match the desired image dimensions(img Shape).
    model.add(Reshape(imgShape))

    #print model summary of the model architecture.
    #model.summary()

    # define the input to the generastor which is the noise vector
    noise = Input(shape = noise_shape)

    # pass the noise vector through the model to generate image 
    img = model(noise)

    # return the generator model which maps noise vector to generated images
    return Model(noise, img)

In [9]:
def discriminator():
    """
    Builds a desciminator model for Generative adversial network (GAN)

    The descriminator acts as a binary classifier that distuingishes between real and fake images.
    It takes an image as input ,flatten it into vector, and processes it through series of fully connected layers with leakyRelu activations. 
    The final layer uses sigmoid activation function output the porbality value representing the validity of the input image.

    Returns:
    Keras.Model : A keras model that maps an input image to validity score(0  to 1)
    
    """

    # initilize a sequential model for descriminators
    model = Sequential()

    # Flatten the input image from its original shape in 1D vector
    # this prepares the image for fully connected layers
    model.add(Flatten(input_shape = imgShape))

    #Add a dense layer with 512 neurons to process the flatten layer
    # this layer helps in learning higher-level features from input
    model.add(Dense(512))

    # add leaky relu activation function to introduce non-linearity.
    #the small negative slope (alpha = 0.2) ensures small gradient for negative inputs.
    model.add(LeakyReLU(alpha=0.2))

    #add another Dense Layer with 256 neurons for further feature extraction
    model.add(Dense(256))

    #add another LeakyRelu activation for non - linearity.
    model.add(LeakyReLU(alpha=0.2))

    # add the output Dense layer with 1 neuron and a single sigmoid activation function.
    #Thje sigmoid function outputs probality between 0 and 1 , representing wheather the input image is real (closer to 1 ) or fake (closer to 0).
    
    model.add(Dense(1,activation = 'sigmoid'))

    # prints the model summary
    #model.summary()

    # defines the input to the descriminator, which is an image with same shape as the generator shape
    img = Input(shape = imgShape)

    #pass the input through the model to get the validity score.
    validity = model(img)

    #return the descriminator model, which maps an input to a validity score.
    return Model(img,validity)
    
    

In [ ]:
# Ensure 'images/' folder exists
os.makedirs("images", exist_ok=True)

# Train Function
def train(epochs, batchSize=128, saveInterval=50):
    # Load the MNIST dataset
    ds_train, ds_info = tfds.load(
        'mnist', 
        split='train',  # Load the training data split
        shuffle_files=True, # Shuffle the files to improve training
        as_supervised=True, # load the data as (image, label) pairs
        with_info=True #also load the dataset metadat like (e.g: , image dimansions)
    )
    
    #Extract only images and ignore labels
    ds_train_images = ds_train.map(lambda image, label: image)

    def normalize_img(image):
        image = tf.cast(image, tf.float32) / 127.5 - 1.0 # normalize from ([0,255] to [-1 to 1])
        image = tf.expand_dims(image, axis=-1) # add an extra channel axis(for gray scale)raining
        return image

    ds_train_images = ds_train_images.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train_images = ds_train_images.cache() # cache the dataset to speed up subsequent epochs
    ds_train_images = ds_train_images.batch(batchSize) # set the batch size
    ds_train_images = ds_train_images.prefetch(tf.data.AUTOTUNE) # prefectc the dat fro faster loading during 

    halfBatch = int(batchSize / 2) # Half batch size for training the discriminator

    for epoch in range(epochs):
        
        
        for real_imgs in ds_train_images:
            # Train Discriminator
            idx = np.random.randint(0, real_imgs.shape[0], halfBatch) # select random half batch for real images
            real_half_batch = tf.gather(real_imgs, idx) # gather the real half batch
    
            noise = np.random.normal(0, 1, (halfBatch, 100)) # generate random noise for fake images
            gen_imgs = generator_model(noise, training=True) # generate fake images from noise
    
            # remove the last dimensions if necessary        
            real_half_batch = tf.squeeze(real_half_batch, axis=-1)

            # train the discriminator on real and fake images
            d_loss_real = discriminator_model.train_on_batch(real_half_batch, tf.ones((halfBatch, 1))) # train on real images
            d_loss_fake = discriminator_model.train_on_batch(gen_imgs, tf.zeros((halfBatch, 1))) # train on fake images

            # take average loss rfom real and fake images
            d_loss = 0.5 * tf.add(d_loss_real, d_loss_fake).numpy()  # Convert to NumPy for easier inspection 

            # withing the same loop train the geenrators by setting the input noise and ultimately
            # training the generator to have the discriminator label its  samples as valid by specifying the gradient loss
            
            
            # Train Generator

            #
            noise = np.random.normal(0, 1, (batchSize, 100)) # generate new noises for training the generator
            valid_y = np.ones((batchSize, 1)) # labels indicating the geenrated images are "real"
            g_loss = combined.train_on_batch(noise, valid_y) # train the generator with labels of real images

            # print losses at specified intervals
            if epoch % 1000 == 0: 
                print(f"Epoch {epoch}:")
                print("[D loss: %f, acc.: %.2f%%] [G loss: %f]" % (d_loss[0], 100 * d_loss[1], g_loss))

        # save generated images at specified intervals
        if epoch % saveInterval == 0:
            save_imgs(epoch)


# Save Images
def save_imgs(epoch):
    r, c = 5, 5 # set the grid size
    noise = np.random.normal(0, 1, (r * c, 100)) # generate the noise for the grid of imagges
    gen_imgs = generator_model.predict(noise) # generate images for the noise input

    gen_imgs = 0.5 * gen_imgs + 0.5 # Rescale the generated images from [-1,1] to [0,1]

    fig, axs = plt.subplots(r, c) # create a grid for platting images
    cnt = 0 # counter for images
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray') # dispaly each generated images in grayscale 
            axs[i, j].axis('off') # hide axis for clean image output
            cnt += 1
    fig.savefig(f"images/mnist_{epoch}.png") # save the image grid to file 
    plt.close() # close the figure to avoid memory issues

# Optimizer
from tensorflow.keras.optimizers import legacy

optimizer = legacy.Adam(learning_rate=0.0002, beta_1=0.5) # define the adam optimizer

# Discriminator


# Build and compile the discriminator first. 
#Generator will be trained as part of the combined model, later. 
#pick the loss function and the type of metric to keep track.                 
#Binary cross entropy as we are doing prediction and it is a better
#loss function compared to MSE or other. 


discriminator_model = discriminator() # initilize the discriminator model
discriminator_model.compile(
    loss="binary_crossentropy",  # use binary cross entropy for binary classification
    optimizer=optimizer, # use adam optimizer
    metrics=['accuracy'] # track accuracy
)

# Generator
generator_model = generator() # Initilize the generator model
generator_model.compile(loss="binary_crossentropy", # use the binary cross entropy loss for the geenrator
                        optimizer=optimizer) # 

# Combined Model
z = Input(shape=(100,)) # inpout noise vector for the generator 
img = generator_model(z) # generate an image from the noise  vector
discriminator_model.trainable = False # freeze the discriminator during generatoe training
valid = discriminator_model(img) # the velidiity of the geenrated image (real or fake)
combined = Model(z, valid) # combine mode : generaotr + discriminator
combined.compile(loss="binary_crossentropy", optimizer=optimizer) # compile the combined model with loss and optimizer

# Train GAN
train(epochs=100000, batchSize=60000, saveInterval=10000) # start thr training process with large abtch size and save every 10k epochs

# Save Generator Model
generator_model.save('generator_model') #save the trained generator model after training

2025-01-05 17:17:10.979423: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43598 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:ce:00.0, compute capability: 8.6
2025-01-05 17:17:15.778225: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 0:
[D loss: 0.661026, acc.: 59.49%] [G loss: 0.615390]
1/1 [==============================] - 0s 138ms/step
Epoch 1000:
[D loss: 0.607098, acc.: 67.38%] [G loss: 0.957948]
Epoch 2000:
[D loss: 0.639631, acc.: 63.76%] [G loss: 0.880633]
Epoch 3000:
[D loss: 0.666328, acc.: 59.41%] [G loss: 0.832610]
Epoch 4000:
[D loss: 0.675556, acc.: 56.42%] [G loss: 0.817644]
Epoch 5000:
[D loss: 0.682835, acc.: 55.08%] [G loss: 0.807822]
Epoch 6000:
[D loss: 0.679432, acc.: 55.48%] [G loss: 0.817320]
Epoch 7000:
[D loss: 0.689530, acc.: 52.34%] [G loss: 0.780299]
Epoch 8000:
[D loss: 0.692777, acc.: 50.27%] [G loss: 0.775519]
Epoch 9000:
[D loss: 0.692275, acc.: 51.22%] [G loss: 0.768376]
Epoch 10000:
[D loss: 0.683460, acc.: 53.49%] [G loss: 0.783809]
1/1 [==============================] - 0s 107ms/step
Epoch 11000:
[D loss: 0.690056, acc.: 51.93%] [G loss: 0.779655]
Epoch 12000:
[D loss: 0.683185, acc.: 54.24%] [G loss: 0.787557]
Epoch 13000:
[D loss: 0.673550, acc.: 57.65%] [G loss: 0.79934